In [3]:
# Imports
import os
import warnings 
import sys
import importlib
import geopandas as gpd
from pathlib import Path
import pandas as pd
from shapely.geometry import Point

directory = Path(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
sys.path.append(str(directory))

modules_to_reload = [
    'src.metric_util_OLD'
]

# Reload and import each module
for module_name in modules_to_reload:
    module = importlib.import_module(module_name)
    importlib.reload(module)
    globals().update({name: getattr(module, name) for name in dir(module) if not name.startswith('_')})

from src.metric_util_OLD import *

from src.config.landobject_config import north_sea_ports
 

warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
# Ignore all warnings
warnings.filterwarnings("ignore")

In [9]:
fp = '/Users/loucas/Documents/ORG/things too big for git/seabed_substrate.geojson'

# Load the AOI
seabed = gpd.read_file(fp)



In [10]:
def classify_substrate(gdf):
    # Define substrate categories
    substrate_categories = {
        'Rocky or till': ['rock', 'igneous rock', 'metamorphic rock', 'granite', 'basalt', 'gneiss', 'schist', 'quartzite', 'till',
                          'andesite', 'peridotite', 'phonolite',  'dacite', 'granitoid', 'quartz diorite', 'diorite', 'tuffite', 'gabbro', 'igneous material',
                          'wacke', 'ash tuff, lapillistone, and lapilli tuff'],  # Added wacke here as it's typically a hard sandstone
        'Mud to muddy Sand': ['clay', 'mud', 'mudstone', 'silt', 'silty sand', 'sandy mud', 'slate', 'shale', 'limestone', 'chalk', 'calcareous carbonate sediment', 'calcareous carbonate sedimentary material'],
        'Sand': ['sand', 'sandstone', 'quartz sand', 'dolomite'],
        'Coarse substrate': ['gravel', 'pebbles', 'cobbles', 'boulders', 'conglomerate'],
        'Mixed sediment': ['mixed sediment', 'sand and gravel', 'muddy gravel', 'sedimentary material'],
        # 'Calcareous sediment': ['limestone', 'chalk', 'calcareous carbonate sediment', 'calcareous carbonate sedimentary material']
    }
    
    def get_substrate(lithology):
        if pd.isna(lithology) or lithology == 'NULL':
            return 'Unknown'
        
        lithology = str(lithology).lower()
        
        for category, materials in substrate_categories.items():
            if any(material in lithology for material in materials):
                return category
        
        print(f"Unclassified lithology: {lithology}")
        return 'Unknown'
    
    # Apply the function to create a new 'Substrate' column
    gdf['Substrate'] = gdf['LithologyValue1'].apply(get_substrate)
    
    return gdf

# Example usage:
gdf = classify_substrate(seabed)
# Dissolve the GeoDataFrame by 'Hardness'
dissolved_gdf = gdf.dissolve(by='Substrate', aggfunc='first')

# Reset the index to make 'Hardness' a column again
dissolved_gdf = dissolved_gdf.reset_index()

gdf_reprojected = dissolved_gdf.to_crs(epsg=3035)

# Save the dissolved GeoDataFrame to a new GeoJSON file
gdf_reprojected.to_file('/Users/loucas/Documents/ORG/things too big for git/seabed_substrate_final.geojson', driver='GeoJSON')

Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified lithology: n/a
Unclassified litholo

# BIRDS

In [4]:
import numpy as np
import rasterio
import geopandas as gpd
from rasterio import features

def create_point_count_raster(gdf, template_raster_path, output_raster_path):
    # Read the template raster
    with rasterio.open(template_raster_path) as src:
        meta = src.meta.copy()
        transform = src.transform
        shape = src.shape

    # Update metadata for the new raster
    meta.update(dtype=rasterio.uint32, count=1, nodata=None)

    # Create a numpy array to store the point counts
    point_count = np.zeros(shape, dtype=np.uint32)

    # Rasterize the points, counting them for each cell
    shapes = ((geom, 1) for geom in gdf.geometry)
    burned = features.rasterize(shapes=shapes, out=point_count, transform=transform, 
                                all_touched=False, merge_alg=rasterio.enums.MergeAlg.add)

    # Write the new raster
    with rasterio.open(output_raster_path, 'w', **meta) as dst:
        dst.write(burned, 1)


# Example usage
bird_fp = '/Users/loucas/Downloads/birds.geojson'
template_raster_fp = '/Users/loucas/Documents/ORG/github/marine-planning/data/rasters/international/wind_speed.tif'
output_raster_path = '/Users/loucas/Downloads/birds.tiff'


gdf = gpd.read_file(bird_fp)

create_point_count_raster(gdf, template_raster_fp, output_raster_path)

# Hub generator

In [1]:
import random
import copy
import math

def generate_hub_configurations(hubs, max_distance_km=20, iterations=1000):
    def modify_coordinate(coord, max_distance_km):
        # Convert max_distance from km to degrees (approximate)
        max_distance_deg = max_distance_km / 111  # 1 degree is approximately 111 km
        return coord + random.uniform(-max_distance_deg, max_distance_deg)

    for _ in range(iterations):
        new_hubs = copy.deepcopy(hubs)
        for hub in new_hubs.values():
            hub['latitude'] = modify_coordinate(hub['latitude'], max_distance_km)
            hub['longitude'] = modify_coordinate(hub['longitude'], max_distance_km)
        yield new_hubs

# Example usage:
hubs = {
    "sorlige nordsjo 2": {"latitude": 57.853, "longitude": 4.748, "capacity": 3},
    "danish": {"latitude": 56.089, "longitude": 5.664, "capacity": 10},
    "german": {"latitude": 54.971, "longitude": 6.264, "capacity": 10},
    "NL": {"latitude": 54.122, "longitude": 3.905, "capacity": 10},
    "PE zone": {"latitude": 51.550, "longitude": 2.470, "capacity": 3.5},
}

generator = generate_hub_configurations(hubs)

In [73]:
import itertools

def generate_all_hub_configurations(hubs, distance_km=20):
    # Convert 20 km to degrees (approximate)
    distance_deg = distance_km / 111  # 1 degree is approximately 111 km

    # Generate the 5 possible locations for each hub
    hub_locations = {}
    for hub_name, hub_data in hubs.items():
        base_lat, base_lon = hub_data['latitude'], hub_data['longitude']
        hub_locations[hub_name] = [
            (base_lat, base_lon),  # Current
            (base_lat + distance_deg, base_lon),  # North
            (base_lat, base_lon + distance_deg),  # East
            (base_lat, base_lon - distance_deg),  # West
            (base_lat - distance_deg, base_lon),  # South
        ]

    # Generate all possible combinations
    hub_names = list(hubs.keys())
    location_combinations = itertools.product(range(5), repeat=len(hub_names))

    for combo in location_combinations:
        new_hubs = copy.deepcopy(hubs)
        for hub_name, location_index in zip(hub_names, combo):
            new_lat, new_lon = hub_locations[hub_name][location_index]
            new_hubs[hub_name]['latitude'] = new_lat
            new_hubs[hub_name]['longitude'] = new_lon
        yield new_hubs

# Example usage:
hubs = {
    "sorlige nordsjo 2": {"latitude": 57.853, "longitude": 4.748, "capacity": 3},
    "danish": {"latitude": 56.089, "longitude": 5.664, "capacity": 10},
    "german": {"latitude": 54.971, "longitude": 6.264, "capacity": 10},
    "NL": {"latitude": 54.122, "longitude": 3.905, "capacity": 10},
    "PE zone": {"latitude": 51.550, "longitude": 2.470, "capacity": 3.5},
}

generator = generate_all_hub_configurations(hubs)

# Generate grid

In [11]:
import rasterio
import geopandas as gpd
from shapely.geometry import LineString

def create_grid_gdf(raster_path):
    # Open the raster file
    with rasterio.open(raster_path) as src:
        # Get raster bounds and resolution
        left, bottom, right, top = src.bounds
        res_x, res_y = src.res

    # Create vertical lines
    vertical_lines = []
    x = left
    while x <= right:
        line = LineString([(x, bottom), (x, top)])
        vertical_lines.append(line)
        x += res_x

    # Create horizontal lines
    horizontal_lines = []
    y = bottom
    while y <= top:
        line = LineString([(left, y), (right, y)])
        horizontal_lines.append(line)
        y += res_y

    # Combine all lines
    all_lines = vertical_lines + horizontal_lines

    # Create GeoDataFrame
    gdf = gpd.GeoDataFrame(geometry=all_lines, crs=src.crs)

    return gdf

# Usage
raster_path = '/Users/loucas/Downloads/big.tif'
grid_gdf = create_grid_gdf(raster_path)

# Save the grid as a shapefile or geopackage
grid_gdf.to_file('/Users/loucas/Documents/ORG/things too big for git/grid.geojson', driver='GeoJSON')  # or use .gpkg for geopackage

In [10]:
def convert_to_gdf(north_sea_ports):
    # List to hold the data for non-substation locations
    non_substations = []

    # Loop through each country in the data
    for country, locations in north_sea_ports.items():
        # Loop through each location
        for location_name, location_data in locations.items():
            if location_data['designation'] != 'substation':
                # Create a dictionary with relevant data
                non_substations.append({
                    'location': location_name,
                    'latitude': location_data['latitude'],
                    'longitude': location_data['longitude'],
                    'designation': location_data['designation'],
                    'country': location_data['country'],
                    'geometry': Point(location_data['longitude'], location_data['latitude'])
                })

    # Convert list of non-substation locations to a GeoDataFrame
    gdf = gpd.GeoDataFrame(non_substations, geometry='geometry', crs='EPSG:4326')
    # Conver to EPSG:3035
    gdf = gdf.to_crs(epsg=3035)

    return gdf

# Example usage:
gdf = convert_to_gdf(north_sea_ports)

# Save the GeoDataFrame to a shapefile or geopackage
gdf.to_file('/Users/loucas/Documents/ORG/things too big for git/ports.geojson', driver='GeoJSON')  # or use .gpkg for geopackage

In [12]:
file = '/Users/loucas/Downloads/iho/iho.shp'

gdf = gpd.read_file(file)
# convert to 3035
gdf = gdf.to_crs(epsg=3035)
gdf.to_file('/Users/loucas/Documents/ORG/things too big for git/NorthSea.geojson', driver='GeoJSON')  # or use .gpkg for geopackage